In [77]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [78]:
def makeUrl(search, start_date, end_date, start_pg=1, end_pg=2):
    date_part = "&pd=3&ds=" + start_date + "&de=" + end_date
    return ["https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + date_part + "&sort=2&start=" + str((i - 1) * 10 + 1) for i in range(start_pg, end_pg + 1)]

In [79]:
def get_naver_articles(urls):
    articles, titles, dates, press = [], [], [], []
    
    for url in urls:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")

        article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
        article_titles = [title.text for title in soup.select(".news_tit")]
        article_infos = [info.text for info in soup.select(".info")]

        new_urls = []
        idx = 0

        while idx < len(article_links) - 1:
            if 'news.naver.com' in article_links[idx]:
                new_urls.append(article_links[idx])
                idx += 1
            elif 'news.naver.com' in article_links[idx + 1]:
                new_urls.append(article_links[idx + 1])
                idx += 2
            else:
                new_urls.append(article_links[idx])
                idx += 1

        # 마지막 요소 처리
        if idx == len(article_links) - 1 and 'news.naver.com' not in new_urls[-1]:
            new_urls.append(article_links[idx])

        date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

        current_dates = []
        current_press = []

        for idx, info in enumerate(article_infos):
            match = date_pattern.search(info)
            if match:
                current_dates.append(match.group(0))
                
                if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
                    current_press.append(article_infos[idx - 1])

        articles.extend(new_urls)
        titles.extend(article_titles)
        dates.extend(current_dates)
        press.extend(current_press)

        time.sleep(1)
        
    return articles, titles, dates, press

In [80]:
def get_article_contents(article_urls):
    contents = []
    article_time = []  # 이름 변경
    for url in tqdm(article_urls):
        news = requests.get(url, headers=headers)
        soup = BeautifulSoup(news.text, "html.parser")
        
        content = soup.select("#dic_area") or soup.select("#articeBody")
        cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content)).replace("flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", '')
        contents.append(cleaned_content)
        
        try:
            date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span").attrs['data-date-time']
        except AttributeError:
            date = re.sub('<[^>]*>', '', str(soup.select_one("#content > div.end_ct > div > div.article_info > span > em")))
        article_time.append(date)  # 이름 변경
    
    return contents, article_time  # 이름 변경


In [82]:
def main():
    search = input("검색할 키워드를 입력해주세요:")
    start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
    end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

    urls = makeUrl(search, start_date, end_date)
    article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)
    
    contents, article_time = get_article_contents(article_urls)  # 이름 변경
    
    df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls })  # 이름 변경
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    return df

if __name__ == "__main__":
    result_df = main()
    print(result_df)

100%|██████████| 20/20 [00:11<00:00,  1.72it/s]

          date    press                                               link  \
0   2017.01.01     전자신문  https://n.news.naver.com/mnews/article/030/000...   
1   2017.01.01   39면 1단  https://n.news.naver.com/mnews/article/030/000...   
2   2017.01.01   울산매일신문                              http://www.iusm.co.kr   
3   2017.01.02   4면 TOP  https://n.news.naver.com/mnews/article/029/000...   
4   2017.01.03     이데일리  https://n.news.naver.com/mnews/article/018/000...   
5   2017.01.03   지디넷코리아  https://n.news.naver.com/mnews/article/092/000...   
6   2017.01.03     IT조선                              http://it.chosun.com/   
7   2017.01.03   아이뉴스24  https://n.news.naver.com/mnews/article/031/000...   
8   2017.01.03     국방일보                        http://kookbang.dema.mil.kr   
9   2017.01.03   A2면 1단  https://n.news.naver.com/mnews/article/015/000...   
10  2017.01.04    오마이뉴스  https://n.news.naver.com/mnews/article/047/000...   
11  2017.01.05     이데일리  https://n.news.naver.com/mnews/article/

In [84]:
#데이터 프레임 저장
filename = '{}_{}_{}.csv'.format(search, start_date.replace(".", ""), end_date.replace(".", ""))
result_df.to_csv(filename, encoding='utf-8-sig', index=False)

In [75]:
search = input("검색할 키워드를 입력해주세요:")
start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

urls = makeUrl(search, start_date, end_date)
article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)

# 여기서 각 리스트의 길이를 확인합니다.
print("Length of article_urls:", len(article_urls))
print("Length of article_titles:", len(article_titles))
print("Length of article_dates:", len(article_dates))
print("Length of article_press:", len(article_press))


contents, article_time = get_article_contents(article_urls)  # 이름 변경

# 길이가 동일한지 확인
if len(article_titles) == len(article_dates)== len(article_press) == len(article_urls) == len(contents):
    df = pd.DataFrame({'title': article_titles, 'dates': article_dates,'press': article_press, 'link': article_urls, 'content': contents, 'time' : article_time})
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    print(df)
else:
    print("The lengths of the lists are not equal. Dataframe cannot be created.")


  0%|          | 0/20 [00:00<?, ?it/s]

Length of article_urls: 20
Length of article_titles: 20
Length of article_dates: 20
Length of article_press: 20


100%|██████████| 20/20 [00:12<00:00,  1.58it/s]

                                              title       dates    press  \
0                           [2017 국제 핫이슈]블록체인 기술 확산  2017.01.01     전자신문   
1                       [신년기획]블록체인, 금융권 넘어 IT기업도 관심  2017.01.01   39면 1단   
2   [2017 신년특집] 세계 금융의 대변화-화폐없는 시대가 온다 디지털 가상화폐 ...  2017.01.01   울산매일신문   
3                 블록체인 등 최신 핀테크 기술 도입… `금융 한류` 일으키자  2017.01.02   4면 TOP   
4                         비트코인, 3년만에 처음으로 1000달러 돌파  2017.01.03     이데일리   
5                       '암호화 화폐' 비트코인, 3년만에 1천달러 돌파  2017.01.03   지디넷코리아   
6                  비즈니스로 진화한 사이버 범죄, ‘한국형 랜섬웨어’ 현실로  2017.01.03     IT조선   
7                             KISA, 암호기술팀·블록체인TF 신설  2017.01.03   아이뉴스24   
8                 교황청의 문장.잉글랜드 국기 모두 전쟁에서 만들어진 ‘심벌’  2017.01.03     국방일보   
9                                   비트코인 1000달러 넘었다  2017.01.03   A2면 1단   
10                          김기춘 위기에 빠트린 사건 세 가지의 결말  2017.01.04    오마이뉴스   
11                        비트코인 사상최고가…中 자본통제에 `풍선효과`  2017.01.05     이데일리   
12          

In [22]:
import pandas as pd
pd.read_csv('/Users/kimkirok/projects/go/비트코인/비트코인_20220101_20230830_1.csv')

,title,dates,press,link,content,time
0,"비트코인 4만8000달러 반등, 1시간 동안 1500달러 급등",2022.01.01,글로벌이코노믹,http://www.g-enews.com/ko-kr/news/article/news...,NaN,NaN
1,[코인시황] 2021년 마지막 날... 비트코인 상승세,2022.01.01,내외경제tv,http://www.nbntv.co.kr/news/articleView.html?i...,NaN,NaN
2,"[2022년 유망코인] 급성장 중인 테라(LUNA), 투자 리스크는 무엇?",2022.01.01,코인리더스,http://coinreaders.com/18131,NaN,NaN
3,새해 ‘NFT 시장’ 거품 빠지고 가치투자 뿌리내릴까,2022.01.01,4면 TOP,http://news.kmib.co.kr/article/view.asp?arcid=...,NaN,NaN
4,"[2022 증시]전문가 84% ""文정부, 부동산 실패""..올해 집값 ↓· 비트코인 ↑",2022.01.01,머니투데이언론사 선정,http://news.mt.co.kr/mtview.php?no=20211231162...,NaN,NaN
5,"[K-Biz 리뷰] LS일렉트릭, 中 전기차 충전소시장 출사표(1월1일)",2022.01.01,더구루,https://www.theguru.co.kr/news/article.html?no...,NaN,NaN
6,"미국증시 마감 5분전...장중 비트코인 · 비트코인ETF, ETHER '2% 이상씩...",2022.01.01,초이스경제,http://www.choicenews.co.kr/news/articleView.h...,NaN,NaN
7,[신년기획-차이나리스크①]'세계의 공장' 흔들…성장률 감소세,2022.01.01,뉴데일리,https://biz.newdaily.co.kr/site/data/html/2021...,NaN,NaN
8,"[2022 블록체인 시대 본격화①] 올해 핵심 키워드 'NFT', 풀어야할 숙제는?",2022.01.01,글로벌경제,http://www.getnews.co.kr/news/articleView.html...,NaN,NaN
9,"뉴욕 증시, 마지막날 이익매물에 하락 마감...다우 0.16%↓ 나스닥 0.61%↓",2022.01.01,뉴시스,http://www.newsis.com/view/?id=NISX20220101_00...,NaN,NaN


In [29]:
import pandas as pd
pd.read_csv('비트코인_20220124_20230830_9.csv')

,title,dates,press,link,content,time
0,"센골드, 2021년 거래액 1,000억 돌파하며 대안 투자처로 각광",2022.01.28,지디넷코리아,https://n.news.naver.com/mnews/article/092/000...,"\n\t\t\t아이티센(대표 박정재, 윤석형)그룹 계열사인 한국금거래소디지털에셋(대...",2022-01-28 10:07:02
1,"비트코인, 4500만원대 횡보 지속…MSTR ""계속 매입할 것""",2022.01.28,SBS Biz,https://n.news.naver.com/mnews/article/374/000...,\n\n\n\n\n비트코인이 4500원대에서 횡보를 이어가고 있습니다.오늘(28일)...,2022-01-28 10:08:01
2,[크립토 브리핑] 4500만원대 횡보하는 비트코인...가상자산과 주식의 상관관계...,2022.01.28,테크M,http://techm.kr,NaN,NaN
3,"3개월째 이어지는 비트코인 조정, 언제쯤 끝날까 [한경 코알라]",2022.01.28,한국경제언론사 선정,https://n.news.naver.com/mnews/article/015/000...,\nWeekend Briefing\n\n\n\n서울 서초구 빗썸 고객지원센터 전광판...,2022-01-28 10:11:01
4,[인포그래픽 뉴스/업비트] 숫자로 보는 오늘의 비트·알트코인 (1월 28일),2022.01.28,코인리더스,http://coinreaders.com,NaN,NaN
...,...,...,...,...,...,...
295,"낙폭 줄이고 있어.. 비트코인 4,620만원 상회.. 이더리움 314만원.. 안갯속...",2022.01.31,매일안전신문,http://peoplesafe.kr,NaN,NaN
296,비트코인 와르르 또 급락 가상화폐 ETF 6건 모두 승인 거절 뉴욕증시 선물 부...,2022.01.31,글로벌이코노믹,http://www.g-enews.com/,NaN,NaN
297,‘가상화폐 줄하락’ 비트코인 4600만원대까지 떨어져,2022.01.31,뷰어스,http://www.theviewers.co.kr,NaN,NaN
298,"비트코인, 4700만원 대에서 하루 만에 4500만원 후반으로 하락",2022.01.31,아주경제,http://www.ajunews.com,NaN,NaN


In [13]:
import pandas as pd
pd.read_csv('비트코인_20220101_20230830.csv')

,title,dates,press,link,content,time
0,"비트코인 4만8000달러 반등, 1시간 동안 1500달러 급등",2022.01.01,글로벌이코노믹,http://www.g-enews.com/,NaN,NaN
1,[코인시황] 2021년 마지막 날... 비트코인 상승세,2022.01.01,내외경제tv,http://www.nbntv.co.kr,NaN,NaN
2,"[2022년 유망코인] 급성장 중인 테라(LUNA), 투자 리스크는 무엇?",2022.01.01,코인리더스,http://coinreaders.com,NaN,NaN
3,새해 ‘NFT 시장’ 거품 빠지고 가치투자 뿌리내릴까,2022.01.01,4면 TOP,https://n.news.naver.com/mnews/article/005/000...,\n2006년 날린 최초의 트윗 34억원이세돌·알파고 겨룬 기보 2.5억원스토리텔링...,2022-01-01 04:03:01
4,"[2022 증시]전문가 84% ""文정부, 부동산 실패""..올해 집값 ↓· 비트코인 ↑",2022.01.01,머니투데이언론사 선정,https://n.news.naver.com/mnews/article/008/000...,\n\n\n\n\n증시 전문가들은 올해 부동산 가격이 서울은 오르지만 나머지 지역에...,2022-01-01 04:31:00
...,...,...,...,...,...,...
95,암호화폐 상속·증여받으면 2개월 평균價 따져 세금 냅니다,2022.01.02,A23면 1단,https://n.news.naver.com/mnews/article/015/000...,\n2022 주식투자 가이드양도세 과세는 내년으로 연기올해부터 코인을 상속·증여받으...,2022-01-02 17:01:01
96,"디파이야, 디파이야…코인 맡길게, 이자 다오~",2022.01.02,A23면 1단,https://n.news.naver.com/mnews/article/015/000...,\n2022 주식투자 가이드디파이 플랫폼이 이자 지급원금보장 안 되고 도난 사례도한...,2022-01-02 17:01:02
97,[코인시황] 비트코인 횡보 속 파워렛저(POWR) 급등,2022.01.02,내외경제tv,http://www.nbntv.co.kr,NaN,NaN
98,"""보험도 가상자산 활용…투자·지급 수단으로""",2022.01.02,A18면 1단,https://n.news.naver.com/mnews/article/009/000...,\n보험연구원 보고서보험산업이 가상자산을 적극 활용해야 한다는 주장이 제기됐다.황인...,2022-01-02 18:09:04


In [12]:
import os
folder_path=os.getcwd()+'/비트코인 2023'

file_list=os.listdir(folder_path)

In [13]:
len(file_list)

63

In [14]:
import pandas as pd
a=pd.read_csv(folder_path+'/'+file_list[0])


In [15]:
for i, c in enumerate(file_list[1:]):
    if c.endswith('.csv'):
        print(i,c)
        a = pd.concat([a, pd.read_csv(os.path.join(folder_path, c))])


0 비트코인_20230118_20230830_4.csv
1 비트코인_20230412_20230830_2.csv
2 비트코인_20230118_20230830_3.csv
3 비트코인_20230118_20230830_2.csv
4 비트코인_20230412_20230830_3.csv
5 비트코인_20230412_20230830_1.csv
6 비트코인_20230118_20230830_1.csv
7 비트코인_20230823_20230830_3.csv
8 비트코인_20230513_20230830_1.csv
9 비트코인_20230823_20230830_2.csv
10 비트코인_20230513_20230830_2.csv
11 비트코인_20230330_20230830_4.csv
12 비트코인_20230724_20230830_4.csv
13 비트코인_20230823_20230830_1.csv
14 비트코인_20230513_20230830_3.csv
15 비트코인_20230101_20230830_4.csv
16 비트코인_20230101_20230830_1-1.csv
17 비트코인_20230724_20230830_1.csv
18 비트코인_20230330_20230830_1.csv
19 비트코인_20230101_20230830_1.csv
20 비트코인_20230101_20230830_1-2.csv
21 비트코인_20230101_20230830_3.csv
22 비트코인_20230513_20230830_4.csv
23 비트코인_20230724_20230830_2.csv
24 비트코인_20230330_20230830_2.csv
25 비트코인_20230330_20230830_3.csv
26 비트코인_20230724_20230830_3.csv
27 비트코인_20230101_20230830_2.csv
28 비트코인_20230811_20230830_1.csv
29 비트코인_20230317_20230830_4.csv
30 비트코인_20230704_20230830_4.csv
31 비트코인_202303

In [16]:
a['dates'] = pd.to_datetime(a['dates'])
sorted_df = a.sort_values(by='dates', ascending=True)

In [17]:
sorted_df.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [18]:
sorted_df.to_csv(os.getcwd()+'/'+'비트코인 통합 2023/'+'비트코인 2023.csv')

In [19]:
import numpy as np

In [20]:
print(2023)
print('전체갯수',len(sorted_df))
print('기사내용 있는 것',len(sorted_df[sorted_df['content'].notna()]))

2023
전체갯수 26899
기사내용 있는 것 8667


In [21]:
sorted_df

,title,dates,press,link,content,time
0,"리플, 연초부터 가상자산 하락하자 동반 추락",2023-01-01,국제뉴스,http://www.gukjenews.com/,NaN,NaN
1,비트코인과 이더리움 차분한 흐름..계묘년에는 제도적 정비와 성투하길..리스...,2023-01-01,매일안전신문,http://peoplesafe.kr,NaN,NaN
2,"비트코인, 2022년 16,587달러로 거래 마감...""올해도 힘든 해 될 것""",2023-01-01,코인리더스,http://coinreaders.com,NaN,NaN
3,"FTX 창업자 샘 뱅크먼-프리드, 다음주 재판 시작",2023-01-01,라디오코리아,http://www.radiokorea.com,NaN,NaN
4,[아듀2022 ②] 머지부터 FTX 파산까지...올해의 주요 이슈 TOP 10,2023-01-01,토큰포스트,https://www.tokenpost.kr,NaN,NaN
...,...,...,...,...,...,...
26894,"[토큰포스트 오후 브리핑] 비트코인 초기 개발자, 비트코인 네트워크에 '드라...",2023-08-23,토큰포스트,https://www.tokenpost.kr,NaN,NaN
26895,"마크 큐반 ”오픈씨, 로열티 시스템 폐지는 큰 실수”",2023-08-23,코인리더스,http://coinreaders.com,NaN,NaN
26896,"'소옆경2' 김래원이 체포해야 할 최종 빌런, '덱스'는 누구인가",2023-08-23,SBS연예뉴스,https://n.news.naver.com/mnews/article/416/000...,\n \n[SBS연예뉴스 | 강선애 기자] '소방서 옆 경찰서 그리고 국과수'의 최...,2023.08.23. 오전 10:45
26897,"[토큰포스트 뉴욕 브리핑] 비트코인 초기 개발자, BTC 사이드체인 도입 제안 ...",2023-08-23,토큰포스트,https://www.tokenpost.kr,NaN,NaN
